In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
edenai_api_key = os.getenv('EDENAI_API_KEY')

# print (openai.api_key)
# print (edenai_api_key)


Als je de print commando's uitvoert, zou je de key(s) die je wilt gebruiken moeten zien, als je `None` ziet staan is er iets mis.

De key van EdenAI is een Bearer token, daar kennen jullie alles van uit de Webservices cursus. Rechtstreeks de EdenAI API aanspreken doe je met een POST request naar de juiste url.
EdenAI heeft véél endpoints, een ChatGPT request zou er als volgt uitzien:

In [ ]:
from enum import Enum

from langchain.llms import EdenAI, OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.embeddings.edenai import EdenAiEmbeddings
from langchain.embeddings import OpenAIEmbeddings

class API(Enum):
    OPEN_AI = 1
    GPT4All = 2
    EDEN_AI = 3
    

def get_llm(which_model=API.OPEN_AI, temperature = 0.0):
    if which_model == API.OPEN_AI:
        # OpenAI heeft ook een ChatModel, maar dat is niet makkelijk transparent te gebruiken als je ook EdenAI wil gebruiken in dezelfde requests. Als je enkel met OpenAI wenst te werken is dit zeker een goede optie. Je zal dan wel het output formaat moeten aanpassen.
        # return ChatOpenAI(temperature=temperature, model="gpt-3.5-turbo")
        return OpenAI(temperature=temperature)
    elif which_model == API.EDEN_AI:
        return EdenAI(edenai_api_key=edenai_api_key,provider="openai", model="gpt-3.5-turbo-instruct", temperature=temperature, max_tokens=250)

def get_embedding(which_model=API.OPEN_AI):
    if which_model == API.OPEN_AI:
        return OpenAIEmbeddings()
    elif which_model == API.EDEN_AI:
        return EdenAiEmbeddings(edenai_api_key=edenai_api_key,provider="openai")
       

In [ ]:
preferred_model = API.OPEN_AI

# llm = get_llm(API.EDEN_AI, 0.0)
# creative_llm = get_llm(API.EDEN_AI, 0.9)
llm = get_llm(preferred_model, 0.0)
creative_llm = get_llm(preferred_model, 0.9)
embeddings = get_embedding(preferred_model)


# Chat with your data

![langchain](img/langchain.jpg)

(image credit: langchain.com)


## Document Loading

Extra data kan in allerlei formaten voorkomen, PDF, JSON, tekst, ... en kan zowel gestructureerd of ongestructureerd zijn.

Om dit te illustreren starten we met een PDF van "The Little Book on Deep Learning" (https://fleuret.org/public/lbdl.pdf), vooral omdat deze een creative commons license heeft.

### Youtube

Maar je kan ook youtube video's als input nemen, we combineren de audio loader van youtube, and de OpenAI whisper parser, die samen een youtube video omzetten in een stuk tekst dat als bron kan dienen.
(om dit te doen werken heb je ook ffmpeg nodig, MacOS/Linux: `brew/apt-get/... install ffmpeg`, Windows: downloaden en prutsen)



In [ ]:
# yt_dlp heeft ffmpeg nodig; dus we zorgen dat het in het standaard path gevonden wordt
# onderstaande paden zijn typisch voor MacOS homebrew, je zal ze waarschijnlijk moeten aanpassen naargelang je OS en waar je ffmpeg geïnstalleerd hebt
import sys
sys.path.append('/opt/homebrew/bin/ffmpeg')
sys.path.append('/opt/homebrew/bin/ffprobe')

In [ ]:

# from yt_dlp.postprocessor import FFmpegPostProcessor
# FFmpegPostProcessor._ffmpeg_location.set('/opt/homebrew/bin/')

In [ ]:
# OPGELET
#
# dit deel uitvoeren duurt snel een aantal minuten; audio transcriben is ook relatief duur $0.006/min of dus $0.36 per uur
# geen extreme bedragen, maar voer dit stuk code ook niet te vaak uit

# Transformer Neural Networks van CodeEmporium
url="https://www.youtube.com/watch?v=TQQlZhbC5ps"


### URL

Je kan ook gewoon alle content van een url inladen, om dan met de data van een website te 'chatten'. De standaard `WebBaseLoader` kan HTML inlezen en beschikbaar maken, dit werkt wel enkel voor sites die niet JavaScript 'heavy' zijn. Om sites die dynamisch opgebouwd worden te kunnen laden heb je een headless browser nodig. Selenium, te gebruiken via `SeleniumURLLoader` is een optie. (hoewel die dingen niet altijd even perfect werken)   

## Document Splitting

We willen de tekst in stukken splitsen, maar dat is veel subtieler dan je zou denken. Je wilt de tekst semantisch splitsen, de stukken zullen later een vector encodering krijgen en gebruikt worden om vragen te beantwoorden. Simpel gezegd wil je bijvoorbeeld niet dat een zin half in één stuk zit, en half in een ander.

Er zijn een aantal splitters voorzien in [langchain](https://python.langchain.com/docs/modules/data_connection/document_transformers/#text-splitters): `CharacterTextSplitter`, `MarkdownHeaderTextSplitter`, `TokenTextSplitter`, `RecursiveCharacterTextSplitter` enz.

De `CharacterTextSplitter` doet ogenschijnlijk niets, dit komt omdat er gesplitst wordt op basis van een specifiek character (standaard een newline), en dus zal een text zonder newlines niet gesplitst worden 


In de praktijk is deze splitter dus niet zo handig, je wilt liefst zo semantisch mogelijk splitsen (per paragraaf of zin) en pas als die paragrafen / zinnen te lang worden, op basis van een ander character.

Daartoe dient de `RecursiveCharacterTextSplitter`, i.p.v. één separator character te gebruiken, geef je een lijst van separators mee, er zal eerst gesplitst worden volgens het eerste character, als de stukken dan nog te lang zijn opnieuw via de tweede, enz.
Tot er geen separators meer over zijn of elk stuk klein genoeg geworden is 

Een andere manier is om te splitsen op basis van tokens. Dit is belangrijk als we met LLM's werken, die een context hebben die beperkt is tot een aantal tokens

Voor markdown files gebruiken we best de `MarkdownHeaderTextSplitter`, zoals de naam suggereert worden markdown files gesplitst op basis van de headers, en de informatie uit die headers komt dan in de metadata terecht.

### vectorstores

De volgende stap is al deze chunks in een vectorstore op te slaan, zodat we snel en makkelijk 'gelijkaardige' content kunnen terugvinden (die we dan samen met onze vraag naar een LLM doorsturen, maar dat komt later)

We dienen eerste vector embeddings te creëren, vector voorstellingen van onze stukken tekst, hiervoor gebruiken we OpenAI

### Chroma

Chroma is een vectorstore die volledig in het geheugen draait, ideaal om snel wat code te runnen (en te demonstreren hier), voor grotere toepassingen bestaan er veel gehoste oplossing ook, langchain heeft bindings voor de meest courant gebruikte.

### Retrieval

Maximum marginal relevance (MMR), als je gewoon de meest gelijkende resultaten zoekt, gebeurt het soms dat je een aantal resultaten terug krijgt die redundant zijn; die eigenlijk allemaal dezelfde inhoud terug geven. MMR kan hier helpen, het algoritme zal naast de relevantie van de resultaten, ook de 'diversiteit' in rekening nemen, en op basis van beide een nieuwe ranking maken. 






SelfQuery, een ander algoritme dat je met langchain kan gebruiken is SelfQuery, het idee is dat je je vraag stelt in 'natuurlijke taal', en dat de LLM zichzelf gebruikt (vandaar de naam) om onderscheid te maken tussen delen van de vraag waarmee de metadata kan gefilterd worden, en de eigenlijk inhoud zelf. 

Als we kijken welke metadata er in het resultaat van onze similarity_search zit

### Vragen beantwoorden

We hebben gezien hoe we data kunnen inladen, hoe we documenten kunnen splitten en in een vectorstore krijgen, en hoe we dan een relevant document kunnen opvragen voor onze vragen.

De volgende stap is nu de LLM deze vraag te laten beantwoorden, met behulp van dit document

#### map reduce

Als de documenten te uitgebreid zijn, zullen ze al snel groter zijn dan de beschikbare context voor LLM's. Een oplossing is om map reduce toe te passen, simpel gezegd zal je de documenten opsplitsen, de vraag naar elk sturen 'mappen', en dan de verschillende antwoorden 'reducen'.

Dit leidt snel tot vrij veel API calls dus we gaan dit hier niet demonstreren. Er zijn voorbeelden en uitleg te vinden op langchain als je dit nodig hebt.


### Chat

Om nu echt te kunnen chatten met de data ontbreekt er nog een deel van de puzzel, we moeten de eerder gegeven antwoorden ook kunnen meenemen in een volgende vraag. Zodat we extra verduidelijking kunnen krijgen, zoals we dat tegenwoordig gewend zijn van chatbots.
